In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

In [2]:
#Calculate the total number of schools
total_schools = len(school_data_complete["School ID"].unique())

#Calculate the total number of students
total_students = len(school_data_complete["Student ID"].unique())

#Calculate the total budget
total_budget = school_data_complete["budget"].unique().sum()

#Calculate the average math score
total_math_score = school_data_complete["math_score"].sum()
avg_math_score = total_math_score/total_students

#Calculate the average reading score
total_read_score = school_data_complete["reading_score"].sum()
avg_read_score = total_read_score/total_students

#Calculate the percentage of students with a passing math score (70 or greater)
total_pass_math = school_data_complete.loc[school_data_complete["math_score"] >= 70, :].count().max()
percent_pass_math = total_pass_math/total_students *100

#Calculate the percentage of students with a passing reading score (70 or greater)
total_pass_read = school_data_complete.loc[school_data_complete["reading_score"] >= 70, :].count().max()
percent_pass_read = total_pass_read/total_students *100

#Calculate the percentage of students who passed math and reading (% Overall Passing)
total_pass_overall = school_data_complete.loc[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70) , :].count().max()
percent_pass_overall = total_pass_overall/total_students *100

#Create a dataframe to hold the District Summary Results
district_summary_df = pd.DataFrame({"Total Schools": [total_schools]
                                    ,"Total Students": total_students
                                    ,"Total Budget": total_budget
                                    ,"Average Math Score": avg_math_score
                                    ,"Average Reading Score": avg_read_score
                                    ,"% Passing Math": percent_pass_math
                                    ,"% Passing Reading": percent_pass_read
                                    ,"% Overall Passing": percent_pass_overall
                                   })

#Format dataframe columns of Total Students and Total Budget
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [3]:
reduced_school_data_df = school_data_complete.loc[:,["school_name", "type", "Student ID", "budget", "size", "math_score", "reading_score"] ]
passed_math_df = reduced_school_data_df.loc[reduced_school_data_df["math_score"] >= 70, :]
passed_reading_df = reduced_school_data_df.loc[reduced_school_data_df["reading_score"] >= 70, :]
passed_both_df = school_data_complete.loc[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70) , :]


grouped_schools_type = reduced_school_data_df.groupby(["school_name", "type"], as_index=False)
grouped_schools = reduced_school_data_df.groupby(["school_name"], as_index=False)
grouped_schools_math = passed_math_df.groupby(["school_name"], as_index=False)
grouped_schools_read = passed_reading_df.groupby(["school_name"], as_index=False)
grouped_schools_both = passed_both_df.groupby(["school_name"], as_index=False)

grouped_schools_student_df = pd.DataFrame(grouped_schools_type["Student ID"].count())
grouped_schools_budget_df = pd.DataFrame(grouped_schools["budget"].max())
grouped_schools_size_df = pd.DataFrame(grouped_schools["size"].max())
grouped_schools_math_df = pd.DataFrame(grouped_schools["math_score"].mean())
grouped_schools_read_df = pd.DataFrame(grouped_schools["reading_score"].mean())

grouped_schools_passmath_df = pd.DataFrame(grouped_schools_math["math_score"].count())
grouped_schools_passread_df = pd.DataFrame(grouped_schools_read["reading_score"].count())
grouped_schools_passboth_df = pd.DataFrame(grouped_schools_both["Student ID"].count())

merge_df = pd.merge(grouped_schools_student_df, grouped_schools_budget_df, on="school_name")
merge_df = pd.merge(merge_df, grouped_schools_size_df, on="school_name")
merge_df = pd.merge(merge_df, grouped_schools_math_df, on="school_name")
merge_df = pd.merge(merge_df, grouped_schools_read_df, on="school_name")
merge_df = pd.merge(merge_df, grouped_schools_passmath_df, on="school_name")
merge_df = pd.merge(merge_df, grouped_schools_passread_df, on="school_name")
merge_df = pd.merge(merge_df, grouped_schools_passboth_df, on="school_name")


merge_df["per_student_budget"] = merge_df["budget"]/merge_df["Student ID_x"]
merge_df["passing_math"] = merge_df["math_score_y"]/merge_df["Student ID_x"]*100
merge_df["passing_read"] = merge_df["reading_score_y"]/merge_df["Student ID_x"]*100
merge_df["passing_both"] = merge_df["Student ID_y"]/merge_df["Student ID_x"]*100


school_summary_df = merge_df.rename(columns={"type": "School Type"
                                             ,"Student ID_x": "Total Students"
                                             ,"math_score_x": "Average Math Score"
                                             ,"reading_score_x": "Average Reading Score"
                                             ,"passing_math": "% Passing Math"
                                             ,"passing_read": "% Passing Reading"
                                             ,"passing_both": "% Overall Passing"
                                            })



#Format dataframe columns of Total Students and Total Budget
school_summary_df["Total School Budget"] = school_summary_df["budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["per_student_budget"].map("${:,.2f}".format)

school_summary_final_df = school_summary_df[["school_name", "School Type", "Total Students", "Total School Budget", "Per Student Budget",
                                      "Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"
                                      ]]
school_summary_final_df = school_summary_final_df.set_index("school_name")
school_summary_final_df.index.name=None

school_summary_final_df


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

In [4]:
top_performers_df = school_summary_final_df.sort_values("% Overall Passing", ascending=False)
top_performers_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [5]:
bottom_performers_df = school_summary_final_df.sort_values("% Overall Passing")
bottom_performers_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [6]:
reduced_grade_mathdata_df = school_data_complete.loc[:,["school_name", "grade", "math_score"] ]

ninth_grade_df = reduced_grade_mathdata_df.loc[reduced_grade_mathdata_df["grade"] == "9th", :]
tenth_grade_df = reduced_grade_mathdata_df.loc[reduced_grade_mathdata_df["grade"] == "10th", :]
eleventh_grade_df = reduced_grade_mathdata_df.loc[reduced_grade_mathdata_df["grade"] == "11th", :]
twelfth_grade_df = reduced_grade_mathdata_df.loc[reduced_grade_mathdata_df["grade"] == "12th", :]


grouped_ninth_grade = ninth_grade_df.groupby(["school_name"], as_index=False)
grouped_tenth_grade = tenth_grade_df.groupby(["school_name"], as_index=False)
grouped_eleventh_grade = eleventh_grade_df.groupby(["school_name"], as_index=False)
grouped_twelfth_grade = twelfth_grade_df.groupby(["school_name"], as_index=False)

grouped_ninth_grade_df = pd.DataFrame(grouped_ninth_grade["math_score"].mean())
grouped_tenth_grade_df = pd.DataFrame(grouped_tenth_grade["math_score"].mean())
grouped_eleventh_grade_df = pd.DataFrame(grouped_eleventh_grade["math_score"].mean())
grouped_twelfth_grade_df = pd.DataFrame(grouped_twelfth_grade["math_score"].mean())


merge_math_df = pd.merge(grouped_ninth_grade_df, grouped_tenth_grade_df, on="school_name", suffixes=("_9th", "_10th"))
merge_math_df = pd.merge(merge_math_df, grouped_eleventh_grade_df, on="school_name")
merge_math_df = pd.merge(merge_math_df, grouped_twelfth_grade_df, on="school_name", suffixes=("_11th", "_12th"))

math_summary_df = merge_math_df.rename(columns={"math_score_9th": "9th"
                                                 ,"math_score_10th": "10th"
                                                 ,"math_score_11th": "11th"
                                                 ,"math_score_12th": "12th" 
                                                })
math_summary_df = math_summary_df.set_index("school_name")
math_summary_df.index.name=None
math_summary_df

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [7]:
reduced_grade_readdata_df = school_data_complete.loc[:,["school_name", "grade", "reading_score"] ]

ninth_grade_df = reduced_grade_readdata_df.loc[reduced_grade_readdata_df["grade"] == "9th", :]
tenth_grade_df = reduced_grade_readdata_df.loc[reduced_grade_readdata_df["grade"] == "10th", :]
eleventh_grade_df = reduced_grade_readdata_df.loc[reduced_grade_readdata_df["grade"] == "11th", :]
twelfth_grade_df = reduced_grade_readdata_df.loc[reduced_grade_readdata_df["grade"] == "12th", :]


grouped_ninth_grade = ninth_grade_df.groupby(["school_name"], as_index=False)
grouped_tenth_grade = tenth_grade_df.groupby(["school_name"], as_index=False)
grouped_eleventh_grade = eleventh_grade_df.groupby(["school_name"], as_index=False)
grouped_twelfth_grade = twelfth_grade_df.groupby(["school_name"], as_index=False)

grouped_ninth_grade_df = pd.DataFrame(grouped_ninth_grade["reading_score"].mean())
grouped_tenth_grade_df = pd.DataFrame(grouped_tenth_grade["reading_score"].mean())
grouped_eleventh_grade_df = pd.DataFrame(grouped_eleventh_grade["reading_score"].mean())
grouped_twelfth_grade_df = pd.DataFrame(grouped_twelfth_grade["reading_score"].mean())


merge_read_df = pd.merge(grouped_ninth_grade_df, grouped_tenth_grade_df, on="school_name", suffixes=("_9th", "_10th"))
merge_read_df = pd.merge(merge_read_df, grouped_eleventh_grade_df, on="school_name")
merge_read_df = pd.merge(merge_read_df, grouped_twelfth_grade_df, on="school_name", suffixes=("_11th", "_12th"))

read_summary_df = merge_read_df.rename(columns={"reading_score_9th": "9th"
                                                 ,"reading_score_10th": "10th"
                                                 ,"reading_score_11th": "11th"
                                                 ,"reading_score_12th": "12th" 
                                                })
read_summary_df = read_summary_df.set_index("school_name")
read_summary_df.index.name=None
read_summary_df

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [12]:
#Create dataframe of needed columns only
school_spending_df = school_summary_df.loc[:, ["per_student_budget", "Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

#Create bins
bins = [0, 585, 630, 645, 680]

#Identify grouping labels
group_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

#Add spending range grouping 
school_spending_df["Spending Range (Per Student)"] = pd.cut(school_spending_df["per_student_budget"], bins, labels=group_labels, include_lowest=True)

#Group the data
school_spending_group = school_spending_df.groupby("Spending Range (Per Student)")

#Create final dataframe of average spending data
school_spending_group_df = school_spending_group.mean()

#Format dataframe columns
school_spending_group_df["Average Math Score"] = school_spending_group_df["Average Math Score"].map("{:.2f}".format)
school_spending_group_df["Average Reading Score"] = school_spending_group_df["Average Reading Score"].map("{:.2f}".format)
school_spending_group_df["% Passing Math"] = school_spending_group_df["% Passing Math"].map("{:.2f}".format)
school_spending_group_df["% Passing Reading"] = school_spending_group_df["% Passing Reading"].map("{:.2f}".format)
school_spending_group_df["% Overall Passing"] = school_spending_group_df["% Overall Passing"].map("{:.2f}".format)

#Organize columns
school_spending_group_df = school_spending_group_df[["Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

#Show summary of scores by spending per student
school_spending_group_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Range (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

In [13]:
#Create dataframe of needed columns only
school_size_df = school_summary_df.loc[:, ["size", "Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

#Create bins
size_bins = [0, 1000, 2000, 5000]

#Identify grouping labels
size_group_labels = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]

#Add size grouping 
school_size_df["School Size"] = pd.cut(school_size_df["size"], size_bins, labels=size_group_labels, include_lowest=True)

#Group the data
school_size_group = school_size_df.groupby("School Size")

#Create final dataframe of average size data
school_size_group_df = school_size_group.mean()

#Format dataframe columns
school_size_group_df["Average Math Score"] = school_size_group_df["Average Math Score"].map("{:.2f}".format)
school_size_group_df["Average Reading Score"] = school_size_group_df["Average Reading Score"].map("{:.2f}".format)
school_size_group_df["% Passing Math"] = school_size_group_df["% Passing Math"].map("{:.2f}".format)
school_size_group_df["% Passing Reading"] = school_size_group_df["% Passing Reading"].map("{:.2f}".format)
school_size_group_df["% Overall Passing"] = school_size_group_df["% Overall Passing"].map("{:.2f}".format)

#Organize columns
school_size_group_df = school_size_group_df[["Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

#Show summary of score by school size
school_size_group_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1000),83.82,83.93,93.55,96.10,89.88
Medium(1000-2000),83.37,83.86,93.60,96.79,90.62
Large(2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

In [14]:
#Create dataframe of needed columns only
school_type_df = school_summary_df.loc[:, ["School Type", "Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

#Group the data
school_type_group = school_type_df.groupby("School Type")

#Create final dataframe of average size data
school_type_group_df = school_type_group.mean()

#Format dataframe columns
school_type_group_df["Average Math Score"] = school_type_group_df["Average Math Score"].map("{:.2f}".format)
school_type_group_df["Average Reading Score"] = school_type_group_df["Average Reading Score"].map("{:.2f}".format)
school_type_group_df["% Passing Math"] = school_type_group_df["% Passing Math"].map("{:.2f}".format)
school_type_group_df["% Passing Reading"] = school_type_group_df["% Passing Reading"].map("{:.2f}".format)
school_type_group_df["% Overall Passing"] = school_type_group_df["% Overall Passing"].map("{:.2f}".format)

#Show summary of scores by school types
school_type_group_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67


# Observable Trends

1. Looking at the "Scores by School Spending" summary, the trend appears to be that schools that spend less per student actually have higher overall testing scores.

2. Looking at the "Scores by School Size" summary, large schools are the least successful in terms of overall passing scores, while small and medium schools are comparable.

3. Looking at the "Scores by School Type" summary, charter schools are more successful in terms of overall passing scores over district schools.